## AWS Marketplace Product Usage Demonstration - Text to Speech Synthesis  Model Package

**eNova Text to Speech Synthesis**

This sample notebook demonstrate how to use TTS sysnthesis using SSML. It demonstrate how to deploy it on Amazon Sagemaker.

**Note**: This is a reference notebook and it cannot run unless you make changes suggested in the notebook.

#### Pre-requisites:
1. **Note**: This notebook contains elements which render correctly in Jupyter interface. Open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
1. Ensure that IAM role used has **AmazonSageMakerFullAccess**
1. To deploy this ML model successfully, ensure that:
    1. Either your IAM role has these three permissions and you have authority to make AWS Marketplace subscriptions in the AWS account used:
        1. **aws-marketplace:ViewSubscriptions**
        1. **aws-marketplace:Unsubscribe**
        1. **aws-marketplace:Subscribe**  
    2. or your AWS account has a subscription to eNova Text to Speech model. If so, skip step: [Subscribe to the model package](#1.-Subscribe-to-the-model-package)

#### Contents:
1. [Subscribe to the model package](#1.-Subscribe-to-the-model-package)
2. [Set up the environment](#2.-Setup-the-environment)
3. [Create the session](#3.-Create-the-session)
4. [Create Model](#4.-Create-model)
5. [Perform batch inference](#5.-Perform-batch-inference)
    1. [Configure the input S3 bucket folder](#A.-Configure-the-input-S3-bucket-folder)
    2. [Deploy the model](#B.-Deploy-the-model)

6. [Clean-up](#Clean-up)
    1. [Delete the model](#A.-Delete-the-model)
    2. [Unsubscribe to the listing (optional)](#B.-Unsubscribe-to-the-listing-(optional))


#### Usage instructions
You can run this notebook one cell at a time (By using Shift+Enter for running a cell).


<a name="1.-Subscribe-to-the-model-package"></a>
## 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page DefenSight Botnet Detector.
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms.
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

In [1]:
model_package_arn = 'arn:aws:sagemaker:us-east-1:822940408628:model-package/enova-text-speech-synthesis'

<a name="2.-Setup-the-environment"></a>
## 2. Set up the environment




In [2]:
import sagemaker as sage
from sagemaker import get_execution_role
from sagemaker import ModelPackage
from sagemaker.serializers import CSVSerializer
import boto3
import json

role = get_execution_role()

# S3 prefixes
common_prefix = "hdts-sagemaker-testing"
batch_inference_input_prefix = "tts/input"
batch_inference_output_prefix = "tts/output"
from sagemaker.predictor import Predictor

<a name="3.-Create-the-session"></a>
## 3. Create the session

The session remembers our connection parameters to Amazon SageMaker. We'll use it to perform all of our Amazon SageMaker operations.

In [3]:
sagemaker_session = sage.Session()

<a name="4.-Create-model"></a>
## 4. Create Model

Now we use the above Model Package to create a model

In [4]:
model_name = 'enova_speech_synthesis'

content_type = 'text/xml'

batch_transform_inference_instance_type = 'ml.m5.large'

In [5]:
def predict_wrapper(endpoint, session):
    return sage.predictor.Predictor(endpoint, session, content_type)

# Create a deployable model from the model package
model = ModelPackage(role=role,
                    model_package_arn=model_package_arn,
                    sagemaker_session=sagemaker_session,
                    predictor_cls = predict_wrapper)

<a name="6.-Perform-batch-inference"></a>
## 6. Perform batch inference

In this section, you will perform batch inference using multiple input payloads together. If you are not familiar with batch transform, and want to learn more, see these links:
1. [How it works](https://docs.aws.amazon.com/sagemaker/latest/dg/ex1-batch-transform.html)
2. [How to run a batch transform job](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-batch.html)

<a name="A.-Configure-the-input-S3-bucket-folder"></a>
### A. Configure the input S3 bucket folder

In [6]:
transform_input_folder = 'input'
transform_input = sagemaker_session.upload_data(transform_input_folder,common_prefix,batch_inference_input_prefix)

<a name="B.-Deploy-the-model"></a>
### B. Deploy the model

In [7]:
transformer = model.transformer(1,batch_transform_inference_instance_type,output_path="s3://"+common_prefix+"/"+batch_inference_output_prefix+"/")
transformer.transform(transform_input,content_type=content_type)
transformer.wait()

INFO:sagemaker:Creating transform job with name: enova-text-speech-synthesis-2023-08-22-05-44-18-767


.....................................Starting the inference server with 2 workers.
[2023-08-22 05:50:30 +0000] [9] [INFO] Starting gunicorn 21.2.0
[2023-08-22 05:50:30 +0000] [9] [INFO] Listening at: unix:/tmp/gunicorn.sock (9)
[2023-08-22 05:50:30 +0000] [9] [INFO] Using worker: sync
[2023-08-22 05:50:30 +0000] [12] [INFO] Booting worker with pid: 12
[2023-08-22 05:50:30 +0000] [13] [INFO] Booting worker with pid: 13
169.254.255.130 - - [22/Aug/2023:05:50:38 +0000] "GET /ping HTTP/1.1" 200 1 "-" "Go-http-client/1.1"
169.254.255.130 - - [22/Aug/2023:05:50:38 +0000] "GET /execution-parameters HTTP/1.1" 404 2 "-" "Go-http-client/1.1"
Length of recieved data is 1180
169.254.255.130 - - [22/Aug/2023:05:50:38 +0000] "GET /ping HTTP/1.1" 200 1 "-" "Go-http-client/1.1"
169.254.255.130 - - [22/Aug/2023:05:50:38 +0000] "GET /execution-parameters HTTP/1.1" 404 2 "-" "Go-http-client/1.1"
Length of recieved data is 1180
2023-08-22T05:50:39.007:[sagemaker logs]: MaxConcurrentTransforms=1, MaxPayloa

<a name="4.-Clean-up"></a>
### 4. Clean-up

<a name="A.-Delete-the-model"></a>
### A. Delete the model

In [8]:
model.delete_model()

INFO:sagemaker:Deleting model with name: enova-text-speech-synthesis-2023-08-22-05-44-17-869


<a name="B.-Unsubscribe-to-the-listing-(optional)"></a>
#### B. Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model.

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.